## Primeiro modelo simples para leitura do ECG no formato ImageFolder

In [3]:
!pip install timm

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 1.4 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 kB 1.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.2/383.2 kB 1.1 MB/s eta 0:00:00ta 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm
import matplotlib.pyplot as plt # For data viz
import pandas as pd
import numpy as np
import sys
from tqdm.notebook import tqdm

In [3]:
## Import dataset

class ECGDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data = ImageFolder(data_dir, transform=transform) # Carrega as imagens padrao com transformacao necessaria
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]
    
    @property
    def classes(self):
        return self.data.classes

In [4]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

In [5]:
dataset = ECGDataset(
    data_dir='../archive/ptbx-dataset-diag/',transform=transform
)

In [6]:
dataset

In [9]:
image, label = dataset[6000]
print(label)
image

2


tensor([[[0.9020, 0.9255, 0.9529,  ..., 0.9725, 0.9725, 0.9569],
         [0.8157, 0.7922, 0.8980,  ..., 1.0000, 1.0000, 0.9804],
         [0.8157, 0.8431, 0.8667,  ..., 1.0000, 1.0000, 0.9804],
         ...,
         [0.9804, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 0.9804],
         [0.9804, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 0.9804],
         [0.9529, 0.9725, 0.9725,  ..., 0.9725, 0.9725, 0.9529]],

        [[0.8157, 0.8275, 0.8510,  ..., 0.8706, 0.8706, 0.8627],
         [0.7412, 0.7098, 0.8118,  ..., 0.8980, 0.8980, 0.8902],
         [0.7490, 0.7686, 0.7882,  ..., 0.9098, 0.9098, 0.9020],
         ...,
         [0.9020, 0.9098, 0.9059,  ..., 0.9059, 0.9059, 0.8980],
         [0.8824, 0.8902, 0.8902,  ..., 0.8902, 0.8902, 0.8824],
         [0.9020, 0.9098, 0.9059,  ..., 0.9059, 0.9059, 0.8980]],

        [[0.8118, 0.8235, 0.8471,  ..., 0.8667, 0.8667, 0.8549],
         [0.7373, 0.7059, 0.8078,  ..., 0.8941, 0.8941, 0.8824],
         [0.7451, 0.7647, 0.7843,  ..., 0.9059, 0.9059, 0.

In [10]:
class ClassificatorECG72Classes(nn.Module):
    def __init__(self, num_classes=6):
        super(ClassificatorECG72Classes, self).__init__()
        # Where we define all the parts of the model
        self.base_model = timm.create_model('efficientnet_b0', pretrained=True)
        #self.base_model = timm.create_model('vit_mediumd_patch16_reg4_gap_256.sbb2_e200_in12k_ft_in1k',num_classes=32)

            
        self.features = nn.Sequential(*list(self.base_model.children())[:-1])

        enet_out_size = 1280        # Make a classifier
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(enet_out_size, num_classes)
        ) # saida como linear 
    
    def forward(self, x):
        # Connect these parts and return the output
        x = self.features(x)
        output = self.classifier(x)
        return output

In [12]:
model = ClassificatorECG72Classes(num_classes=6)
print(str(model)[:500])

ClassificatorECG72Classes(
  (base_model): EfficientNet(
    (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bi


In [13]:
# Loss function
criterion = nn.CrossEntropyLoss()
# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [14]:
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)


In [15]:
# Simple training loop
num_epochs = 5
train_losses, val_losses = [], []

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = ClassificatorECG72Classes(num_classes=53)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc='Training loop'):
        # Move inputs and labels to the device
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * labels.size(0)
    train_loss = running_loss / len(train_loader.dataset)
    train_losses.append(train_loss)
    
    # Validation phase
    model.eval()
    running_loss = 0.0
    print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {train_loss}")

Training loop:   0%|          | 0/524 [00:00<?, ?it/s]

KeyboardInterrupt: 